In [310]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [311]:
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.language import Language
from typing import List, Dict, Tuple, Optional

from  src.pipeline.create_pipline_init import create_nlp_pipeline

import json
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

def _load_gaz_file(path: str) -> List[str]:
    with open(path, "r") as gaz_file:
        return [line.strip() for line in gaz_file.readlines()]

def pre_defined_patterns():
    storage_pattern = [
        {"LIKE_NUM": True},
        {"LOWER": {"IN": ["gb", "mb"]}}
    ]

    weights_pattern = [
        {"LIKE_NUM": True},
        {"LOWER": {"IN": ["g", "kg", "grams", "kilograms", "lb", "lbs", "pounds"]}}
    ]

    dimension_pattern = [
        {"LIKE_NUM": True},
        {"LOWER": "x"},
        {"LIKE_NUM": True},
        {"LOWER": "x"},
        {"LIKE_NUM": True}

    ]

    speed_pattern = [{"LIKE_NUM": True}, {"LOWER": {"IN": ["mhz", "ghz"]}}]

    patterns = [
        {"label": "Unit_Data_Storage", "pattern": storage_pattern},
        {"label": "Dimensions", "pattern": dimension_pattern},
        {"label": "Unit_Weights", "pattern": weights_pattern},
        {"label": "Unit_Speed", "pattern": speed_pattern}
    ]
    
    

    return patterns

GAZETTEERS_PATH = "../../data/gazetteers/"


nlp = spacy.load("en_core_web_sm")#en_core_web_lg
storage_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": {"IN": ["gb", "mb"]}}
]


weights_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": {"IN": ["g", "kg", "grams", "kilograms", "lb", "lbs", "pounds"]}}
]


dimension_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": "x"},
    {"LIKE_NUM": True},
    {"LOWER": "x"},
    {"LIKE_NUM": True},
    {"LOWER": "inches"}
     
]

speed_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": {"IN": ["mhz", "ghz"]}}
     
]


ruler = EntityRuler(nlp, overwrite_ents=True)#phrase_matcher_attr="LOWER",
patterns = [{"label": "ORG", "pattern": "Apple"},
            {"label": "Component", "pattern": [{"LOWER": "screen"}]},
            {"label": "Component", "pattern": [{"LOWER": "keyboard"}]},
           {"label": "Unit_Data_Storage", "pattern":storage_pattern},
            {"label": "Dimensions", "pattern":dimension_pattern},
            {"label": "Unit_Weights", "pattern":weights_pattern},
            {"label": "Unit_Speed", "pattern":speed_pattern}
           ]

ruler.add_patterns(patterns)

ruler.to_disk("/data2/zhanghc/RE/low-resource/src/data/ruler/patterns.jsonl")

#ruler.add_patterns([{"label": "COMPONENT", "pattern": "screen"}])
nlp.add_pipe(ruler)

ruler = EntityRuler(nlp)





#PATTERNS_PATH = "../../data/patterns.jsonl"

nlp = create_nlp_pipeline(gazetteers_path=GAZETTEERS_PATH)



In [312]:
from src.pipeline.entity_ruler import create_entity_ruler_phrases
import os

nlp = spacy.load("en_core_web_sm")

gazetteer_phrases = []
if GAZETTEERS_PATH is not None:
    for filename in os.listdir(GAZETTEERS_PATH):
        if filename.endswith(".gaz"):
            label = os.path.splitext(filename)[0].upper()
            gazetteer_phrases += [(phrase, label) for phrase in
                                  _load_gaz_file(os.path.join(GAZETTEERS_PATH,
                                                              filename))]



In [313]:
with nlp.disable_pipes(*[name for name, _ in nlp.pipeline]):
    ruler = create_entity_ruler_phrases(nlp=nlp,
                                              gazetteer_phrases=gazetteer_phrases)
    #ruler = EntityRuler(nlp, overwrite_ents=True)  # phrase_matcher_attr="LOWER",
    #ruler.add_patterns(pre_defined_patterns())

nlp.add_pipe(ruler)


weights_pattern = [
    {"LIKE_NUM": True},
    {"LOWER": {"IN": ["g", "kg", "grams", "kilograms", "lb", "lbs", "pounds"]}}
]
patterns = [{"label": "QUANTITY", "pattern": weights_pattern}]

with nlp.disable_pipes(*[name for name, _ in nlp.pipeline]):
    ruler = EntityRuler(nlp, overwrite_ents=True)  #phrase_matcher_attr="LOWER", patterns = [{"label": "ORG", "pattern": "Apple"}, {"label": "Component", "pattern": [{"LOWER": "screen"}]}, {"label": "Component", "pattern": [{"LOWER": "keyboard"}]}, {"label": "Unit_Data_Storage", "pattern":storage_pattern}, {"label": "Dimensions", "pattern":dimension_pattern}, {"label": "Unit_Weights", "pattern":weights_pattern}, {"label": "Unit_Speed", "pattern":speed_pattern} ]

    ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

In [314]:

#text="""Amazingly small and light, the Sony Vaio PCG-Z1WA notebook PC is ready to hit the road with its bright 14.1-inch Screen, 1.7 GHz battery-saving Centrino processor (for a battery life of up to 6.5 hours), and built-in 54g wireless LAN for connecting to your e-mail and the Web as you roam the hotspots. This elegantly designed laptop also features a 60 GB hard drive, 512 MB of installed RAM, integrated Bluetooth connectivity, and a combo DVD-ROM/CD-RW drive.The BasicsHard Drive:Though not world-beating in size, the 60 GB enhanced-IDE hard drive offers more than enough storage capacity for road warriors.Processor:The 1.7 GHz Pentium M processor is at the top of the Centrino speed range. Because of its design, the Intel Pentium M gets more done per clock tick than a Mobile Intel Pentium 4, thus providing the discrepancy in GHz measurement (i.e., 1.7 versus 3.2). The Pentium M processor is designed specifically for low-power mobile usage, and it features the latest in power-saving technologies--ensuring you push your battery and productivity to the max.Memory:The PCG-Z1WA comes with a 512 MB built-in memory (1 x 512 MB DDR SDRAM, 266 MHz)--a good start for power users. It has a maximum memory expansion up to 1 GB (2 x 512 MB), which is a bit smaller than other notebooks, but understandable due to the PCG-Z1WA's size. It also has an L2 cache of 1 MB and a front-side bus (FSB) speed of 400 MHz.CD/DVD Drive:It comes equipped with a combo DVD/CD-RW drive, which enables you to play DVD-ROM discs and DVD movies as well as burn data and record music to CD discs. You can read and burn CDs at 24x speed, rewrite at 10x, and read; it has an 8x DVD read speed.Keyboard & Mouse:This notebook has an 83-key QWERTY keyboard (with 2mm stroke and 18.5mm pitch) and electro-static touchpad. It features two programmable buttons (one pre-set as maximum brightness and the other as sound mute).Sound and GraphicsThe PCG-Z1WA features an ATI Mobility Radeon 7500 video/graphics card that has 16 MB of DDR SDRAM and offers dual display support (which can work in tandem with the notebook's native display). Its display of 3-D graphics is decent, but certainly not overpowering. The 14.1-inch SXGA+ TFT display has a native resolution of 1400 x 1050 pixels. It also has an sound card that's integrated into the main board, and offers built-in stereo speakers.Connectivity and Expansion SlotsThe PCG-Z1WA has an Intel Pro-Wireless 2200BG integrated wireless LAN that connects using the 802.11g wireless standard. It also offers Bluetooth wireless connectivity, for communicating with peripherals like keyboards, mice, printers, and more. You get a smaller offering of connectivity options (due to the notebook's size):2 USB 2.0 ports for connecting a wide range of peripherals--from digital cameras to MP3 players1 FireWire (also known as IEEE 1394 or i.Link) ports for connecting digital video camcorders and other peripherals1 RGB monitor port1 PCMCIA (or PC Card) slot (Type I/II)1 headphone jack and 1 microphone jackMemory Stick memory card reader (compatible with Memory Stick Pro)It also has an integrated 56K modem (V.92), and an RJ-45 LAN network port (for a 10/100 Ethernet connection to networks and DSL/cable modems).Preloaded System and SoftwareThe PCG-TR3A comes with the Windows XP Home Edition operating system, and it also features Microsoft Works (for working with word processing and spreadsheet documents). It also is loaded with a bevy of Sony multimedia software: PictureGear Studio and Network Smart Capture (digital photo), DVgate Plus (digital video), and SonicStage (digital music). Other preloaded software includes InterVideo WinDVD, Screenblast ACITD and SoundForge, and RealPlayer One.Dimensions and WeightAt 12.4 x 9.7 x 1.5 inches (L x W x H), this is a seriously thin notebook. And it weighs just 4.7 pounds with the battery.PowerThe included rechargeable lithium-ion battery offers between 3 and 5.5 hours of life with typical usage. An optional larger-capacity battery will provide double that (between 6 and 11 hours of battery life).What's in the BoxThis package contains the Sony Vaio PCG-Z1WA notebook PC, rechargeable lithium-ion battery, AC adapter, and operating instructions."""
texta="""Amazingly small and light, 10 lbs the Sony Vaio PCG-Z1WA notebook PC is ready to hit the road with its bright 14.1-inch Screen, 1.7 GHz battery-saving Centrino processor (for a battery life of up to 6.5 hours), and built-in 54g wireless LAN for connecting to your e-mail and the Web as you roam the hotspots."""
doc = nlp(texta)
displacy.render(doc, style="ent", jupyter=True)